<a href="https://colab.research.google.com/github/swKyungbock/2023MLwithTextData/blob/main/7%EC%9E%A5_1_120%EB%8B%A4%EC%82%B0%EC%BD%9C%EC%9E%AC%EB%8B%A8_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## '120다산콜재단' 토픽 모델링
- 비지도학습


## 라이브러리 로드

In [ ]:
# 필요 라이브러리를 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드

In [ ]:
df = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

In [ ]:
df.head(3)

## 문서 만들기
* 제목과 내용을 함께 사용

In [ ]:
df["문서"] = df["제목"] + " " + df["내용"]

In [ ]:
df.head()

## 벡터화

* BOW
* CountVectorizer



In [ ]:
# 단어들의 출현 빈도(frequency)로 여러 문서들을 벡터화하기 위해 CountVectorizer 사용
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=["돋움", "경우", "또는"])

### 참고: fit, transform, fit_transfrom의 차이점
- fit(): 원시 문서에 있는 모든 토큰의 어휘 사전을 배운다
- transform(): 문서를 문서 용어 매트릭스로 변환, transform 이후엔 매트릭스로 변환되어 숫자형태로 변경
- fit_transform(): 어휘 사전을 배우고 문서 용어 매트릭스를 반환, fit 다음에 변환이 오는 것과 동일하지만 더 효율적으로 구현


In [ ]:
# fit_transform을 사용하여 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성
dtm_cv = cv.fit_transform(df["문서"])

In [ ]:
# cv.vocabulary_ 를 봅니다.
cv.vocabulary_

In [ ]:
cv_cols = cv.get_feature_names_out()

In [ ]:
#단어 가방에 있는 모든 단어를 행렬값으로 변환
# '있는','있습니다'와 같은 고빈도 단어를 불용어로 다룰 것인지 고려하기
pd.DataFrame(dtm_cv.toarray(), columns=cv_cols).sum().sort_values()

## 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

* API documentation: https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [ ]:
# 정답인 '분류'의 유일한 값을 확인하여 주제 수를 확인
df["분류"].value_counts()

In [ ]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옴
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수(기본값=10)

from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 10
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42)

In [ ]:
# LDA_model 에 dtm_cv 를 넣어 학습
LDA_model.fit(dtm_cv)

## TF-IDF(Term Frequency - Inverse Document Frequency)

## TfidfVectorizer



In [ ]:
# TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩하여 벡터화하기 위해 TfidfVectorizer를 사용

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=["돋움", "경우", "또는", "있습니다", "있는", "합니다"])
tfidf

In [ ]:
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성
dtm_tfidf = tfidf.fit_transform(df["문서"])

In [ ]:
# tfidf.vocabulary_
cols_tfidf = tfidf.get_feature_names_out()

In [ ]:
# dtm_tf를 axis=0(수직 방향으로) 기준으로 합계를 낸 dist 변수를 생성
# dist 변수를 vocabulary_ 순으로 정렬하여 비율을 확인
dist = np.sum(dtm_tfidf, axis=0)
pd.DataFrame(dist, columns=cols_tfidf).T.sort_values(by=0).tail(10)

In [ ]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 가중치를 적용한 vector를 확인
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인
pd.DataFrame(dtm_tfidf.toarray(), columns=cols_tfidf)

## 코사인 유사도
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(dtm_tfidf[0] , dtm_tfidf)
result_list = similarity_simple_pair.tolist()[0]

In [ ]:
df["유사도"] = result_list
df[["분류", "제목", "유사도"]].sort_values(by="유사도", ascending=False).head(10)